$\newcommand{\xv}{\mathbf{x}}
\newcommand{\Xv}{\mathbf{X}}
\newcommand{\yv}{\mathbf{y}}
\newcommand{\zv}{\mathbf{z}}
\newcommand{\av}{\mathbf{a}}
\newcommand{\Wv}{\mathbf{W}}
\newcommand{\wv}{\mathbf{w}}
\newcommand{\tv}{\mathbf{t}}
\newcommand{\Tv}{\mathbf{T}}
\newcommand{\muv}{\boldsymbol{\mu}}
\newcommand{\sigmav}{\boldsymbol{\sigma}}
\newcommand{\phiv}{\boldsymbol{\phi}}
\newcommand{\Phiv}{\boldsymbol{\Phi}}
\newcommand{\Sigmav}{\boldsymbol{\Sigma}}
\newcommand{\Lambdav}{\boldsymbol{\Lambda}}
\newcommand{\half}{\frac{1}{2}}
\newcommand{\argmax}[1]{\underset{#1}{\operatorname{argmax}}}
\newcommand{\argmin}[1]{\underset{#1}{\operatorname{argmin}}}$

# Reinforcement Learning in Pong
## Final Report

*Andres Calderon Jaramillo*

In this project, I designed and implemented a reinforcement-learning-based controller for a simplified version of <b>Pong</b>. I used Q learning with a neural network approximator to train the agent. After much trial and error, I found a set of parameters and a network topology that was able to learn to control one of the paddles in the game. It successfully competes against a skilled human player. All code was implemented in Java.

## Introduction

In the previous semester, I (together with a teammate) attempted to build a reinforcement-learning-based player for <b>Ludo</b>, a chance board game that has a large state space. Unfortunately, the developed agent was unable to successfully learn a winning policy. I found that reinforcement learning parameters are extremely difficult to tune. The fact that a neural network approximator was used made it even more difficult.

Hence my motivation to try reinforcement learning in a different context. I chose <b>Pong</b> because it is a game that has simple dynamics. This allowed me to focus on tuning the parameters.

To design the controller, I chose Q learning with a neural network approximator for the Q function. I aimed to answer the following questions:

<ul>
  <li>Can an agent that is trained using Q learning behave well (in a robust and stable manner) in a simplified version of Pong?</li>
  <li>How complex does the neural network approximator have to be in order to obtain good learning performance?</li>
  <li>How does the behavior displayed by the trained controller compare to the behavior displayed by a human player?</li>
  <li>Can the agent transfer what it learned in order to play under slightly different game dynamics?</li>
</ul>

## Methods

Pong is a simple two-player game that consists in two paddles and a ball as shown below (image taken from <a href="https://en.wikipedia.org/wiki/Pong" target="_blank">Wikipedia</a>).

<img src="images/pong_wikipedia.png" alt="Pong" />

The goal is for each player to prevent the ball from hitting the player's wall (by using the paddle) while attempting to make the other player miss the ball. In this project, I simplified the gameplay as follows:

<ul>
  <li>When the ball hits a paddle, the angle of refelction is equal to the incoming angle. In the real Pong, the angle of reflection depends on the spot where the ball hits the paddle.</li>
  <li>The speed of the ball remains constant throughout the game.</li>
</ul>

To make the game interesting, each player is allowed to control the vertical acceleration of the paddle. A zero acceleration causes the paddle to keep its current velocity. I implemented all game dynamics from scratch in order to have maximum control over them.

To design the AI agent, I used Q learning. In this algorithm, we update a Q function that outputs the <i>quality</i> of an action applicable to a specific state. Updates to the function occur when the agent gets a reward from the environment. In my implementation, the environment notifies the agent every time the world changes, i.e., at every timestep. The agent can then collect rewards/punishments and make a choice on what action to take next.

To use Q learning I had to define the following:

### State Representation

At every timestep, the agent has access to the entire world. However, only the following information is used to represent the relevant state of the world:

<ul>
  <li>Position of the controller&#39;s paddle - $p_y$ (vertical distance from the top of the world)</li>
  <li>Velocity of the controller&#39;s paddle - $p_{v_y}$ (positive if moving down, negative if moving up)</li>
  <li>Acceleration of the controller&#39;s paddle - $p_{a_y}$ (positive if downward, negative if upward)</li>
  <li>Position of the ball ($x$ coordinate) - $p_x$</li>
  <li>Position of the ball ($y$ coordinate) - $p_y$</li>
  <li>Velocity of the ball (horizontal) - $b_{v_x}$</li>
  <li>Velocity of the ball (vertical) - $b_{v_y}$</li>
</ul>

Such a state should capture all the information needed for the agent to make a decision on how to accelerate the paddle next.

Note that I am using the coordinate system that is conventional in computer programs: the origin is the top-left corner of the world. The positive $x$ direction is towards the right of the screen. The positive $y$ direction is towards the bottom of the screen.

### Action Representation

The action the agent can take is simply the acceleration to apply to the paddle. In order to avoid dealing with a continuous action space, I allowed the agent to apply the following accelerations:

<ul>
  <li>+0.25 - apply a downward acceleration</li>
  <li>0.00 - do not a apply an acceleration (keep the current velocity)</li>
  <li>-0.25 - apply an upward acceleration</li>
</ul>

In my implementation, the agent decides to take one of these three actions at every world update (at every timestep). If the agent applies a correct sequence of actions, it can finely control the speed of the paddle.

### Q Function Approximator

Because the state representation is essentially continuous, the traditional table representation of the Q function is unsuitable. Instead, I used a neural network. The network has eight input units: seven for the state and one for the action. It has one output unit: the quality of the action.

I experimented with numerous topologies. Initially, I tried a one-hidden-layer network, but it did not produce good results. I then attempted to train a network until convergence for a single batch and tried different topologies and activation functions until I found one that trained relatively fast and achieved a low error. The resulting topology is relatively deep:

<ul>
  <li><b>Input layer:</b> 8 units</li>
  <li><b>Hidden layer 1:</b> 21 units, log activation function</li>
  <li><b>Hidden layer 2:</b> 14 units, log activation function</li>
  <li><b>Hidden layer 3:</b> 7 units, log activation function</li>
  <li><b>Hidden layer 4:</b> 5 units, log activation function</li>
  <li><b>Hidden layer 5:</b> 4 units, log activation function</li>
  <li><b>Hidden layer 6:</b> 3 units, log activation function</li>
  <li><b>Output layer:</b> 1 unit, linear activation function</li>
</ul>

To train the neural network, I used <a href="http://www.heatonresearch.com/encog/" target="_blank">Encog</a>, a high-performing Java library that is able to use multiple processor cores. This library provides a log activation function:

$$f(x) = \log (1 + x) \text { if } x \ge 0, \log (1 - x) \text { otherwise}$$

<img src="images/log_activation_function.png" alt="Log Activation Function" />

The equation and the plot were taken from the <a href="https://s3.amazonaws.com/heatonresearch-books/free/Encog3Java-User.pdf" target="_blank">Encog User Guide</a>. According to this guide, the log activation function can be useful to prevent saturation. This can be seen easily in the plot: the derivative does not become zero.

In order to train the network, the Encog documentation recommends the <i>Resilient Propagation</i> (RPROP) algorithm. The following description is based on the user guide:

RPROP is similar to the <i>Manhattan Update Rule</i>. This rule works by checking the sign of the gradient. It updates weight values by a constant. This contrasts with backpropagation, where the amount by which weights are updated depends on the magnitude of the gradient. The Manhattan Update Rule can prevent weights from being updated too drastically. RPROP has a slightly different approach: it can apply different delta values to each weight matrix, and the delta values can change from iteration to iteration depending on the magnitude of the gradient.

I tried different training algorithms built into the library: standard backpropagation, scaled conjugate gradient, the Manhattan Update Rule, and RPROP. RPROP yielded the best performance.

### Training Episodes and Rewards

In spite of Pong being a two-player game, I decided to make the training scenario very simple. The agent is to play against a wall instead of an opponent paddle. The goal is to teach the agent to not miss the ball. This effectively teaches the agent to be defensive. This is different from the real goal in Pong: to hit the ball in such a way that the opponent is likely to miss it.

A training episode starts with the agent's paddle on the left side (in the middle). The ball starts somewhere on the right side of the world. The initial angle of the ball is picked randomly between 120 and 240 degrees (so that it goes toward the agent). Recall that the ball's speed is constant. A training episode ends in one of the following situations:

<ul>
  <li><b>The agent misses the ball:</b> in this case the agent gets a negative reward.</li>
  <li><b>The agent hits the ball a certain number of times:</b> in my implementation, I set this to 10. This rule is in place in order to prevent the episode from extending for too long.</li>
</ul>

Rewards are collected at every timestep as follows:

<ul>
  <li>The agent gets a negative reward that is proportional to the vertical distance between the middle of the paddle and the middle of the ball at the time the ball was missed. I defined the constant of proportionality to be 0.5.</li>
  <li>The agent gets a positive reward of 10.0 if it hits the ball.</li>
  <li>The agent gets a reward of 0.0 in any other case.</li>
</ul>

The neural network is not updated after every episode. Instead, batches are collected. Each batch contains a number of inputs to the neural network (last state and action) and the corresponding target Q values. The target value for a state-action pair $(s_{last}, a_{last})$ depends on whether a random action was chosen in the new state $s_{new}$ according to the $\epsilon$-greedy strategy. If a random action $a_r$ was chosen, the target value is:

$$Q(s_{last}, a_{last}) = reward + \gamma Q(s_{new}, a_r)$$

If a random action was not taken (the agent chose the best action), the target is:

$$Q(s_{last}, a_{last}) = reward + \gamma \max_{a} Q(s_{new}, a)$$

## Results

First, the agent must be trained. The following parameters were found to work well (after a significant amount of trial and error in tuning):

<ul>
  <li><b>Number of training episodes:</b> 40,000</li>
  <li><b>Number of training episodes per batch:</b> 10</li>
  <li><b>Maximum $\epsilon$ (for the $\epsilon$-greedy strategy):</b> 0.9</li>
  <li><b>Minimum $\epsilon$:</b> 0.0</li>
  <li><b>Number of episodes to reach minimum $\epsilon$ ($\epsilon$ decreases linearly):</b> 10,000</li>
  <li><b>Maximum number of ball hits per episode:</b> 10</li>
  <li><b>Number of RPROP iterations per batch:</b> 1</li>
</ul>

We must first compile the Java program. I tested this in one of the CS120 lab computers (`javac 1.8.0_91`).

In [3]:
!cd java_basic && javac -cp encog-java-core-master.jar:. PongUI.java

Next, we can run the training command:

In [4]:
!cd java_basic && java -cp encog-java-core-master.jar:. PongUI train none nn_basic.eg

Batch   Epsilon NN Error Mean Miss Distance
0       0.900   NaN      34.919            
1       0.899   11.448   10.361            
2       0.898   13.984   17.918            
3       0.897   18.054   16.180            
4       0.896   7.835    42.179            
5       0.896   8.973    39.334            
6       0.895   15.780   50.834            
7       0.894   11.532   24.675            
8       0.893   6.124    16.501            
9       0.892   6.818    46.440            
10      0.891   16.582   43.946            
11      0.890   4.121    53.028            
12      0.889   14.992   51.936            
13      0.888   5.158    43.206            
14      0.887   8.521    40.348            
15      0.887   9.462    48.548            
16      0.886   6.623    43.311            
17      0.885   13.382   36.364            
18      0.884   24.687   59.722            
19      0.883   19.539   31.419            
20      0.882   7.500    37.052            
21      0.881   8.626    38.692 

This command trains the agent (neural network weights are randomly initialized) and outputs learning progress information. The second column shows the value of $\epsilon$ for the last episode in each batch. The third column shows the training error as reported by Encog after performing RPROP for the batch. The fourth column is the most important. It shows the average distance by which the agent's paddle missed the ball. The average is calculated over 75 test episodes in which the agent plays by itself agains a wall. Hits count as a missed distance of 0.0. The goal is for this distance to approach 0.0.

The command saves the following files (neural network representations in the Encog format):

<ul>
  <li><b>nn_basic.eg_0</b>: the neural network as it was initialized before training started.</li>
  <li><b>nn_basic.eg_batch_XX</b>: the neural network as it is after batch XX (currently generated every 1000 batches).</li>
  <li><b>nn_basic.eg</b>: the neural network after training (it is updated after every batch).</li>
  <li><b>nn_basic.eg_best</b>: the best neural network found in the entire process (the one that produced the smallest average missed distance).</li>
</ul>

Note that for this notebook, I stopped the agent at batch 1,722 to save time since the performance seemed to be pretty stable toward the end.

Let us look into what the agent learned. First, let us play against it using the neural network before training started. I developed a GUI for the game that allows a human player to interact with the agent. You can run the following cell (when I tried it in a CS120 lab machine, it opened the GUI). Note that I made a backup of the neural network files in case you re-run the training process and it does not produce the same results. If you re-run the training process and wish to use the new files, delete the `nn_bak/` prefix in the commands below.

To control the acceleration of the right paddle, use the up/down arrows. The blue arrows that appear in the paddles represent the acceleration induced by the controller (human or AI).

In [17]:
!cd java_basic && java -cp encog-java-core-master.jar:. PongUI play nn_bak/nn_basic.eg_0 none

We can see that the agent exhibits unsatisfactory behavior. In fact, it tends to like the top of the world. Now, let us see what it does when using the neural network from batch 1,000 (average miss distance = 2.551).

In [18]:
!cd java_basic && java -cp encog-java-core-master.jar:. PongUI play nn_bak/nn_basic.eg_batch_1000 none

We can see that the agent performs much better. If you played for long enough, you may have realized that it is still hesitant in some situations. Let us now try the neural network as it was when I interrupted the process:

In [19]:
!cd java_basic && java -cp encog-java-core-master.jar:. PongUI play nn_bak/nn_basic.eg none

This agent seems very professional!

The most interesting observation to be made here is that the agent learned a great strategy to not miss the ball: follow it. In the last two runs, we can see that the agent attempts to track the trajectory of the ball when it approches the left side. This is so even though the agent seems to still prefer the top of the world (probably because of the initial neural network weights).

Let us see a visualization of the learning process based on the training process output:

<img src="images/avg_miss_distance_by_batch.png" alt="Average Miss Distance by Batch" />

<img src="images/rprop_training_error_by_batch.png" alt="RPROP Training Error by Batch" />

Except for some transient peaks, the average miss distance displays a slowly decreasing behavior. This is close to the ideal behavior of a Q learning process. Similarly, the RPROP training error followed a slowly decreasing pattern. This suggests that the neural network is learning and converging toward the correct Q function.

Let us now try something risky: let us use the final neural network from the training process above but change the game dynamics slightly: increase the ball speed by 2 pixels/timestep. First, we compile the new version:

In [20]:
!cd java_enhanced && javac -cp encog-java-core-master.jar:. PongUI.java

Now, let us try it:

In [21]:
!cd java_enhanced && java -cp encog-java-core-master.jar:. PongUI play nn_bak/nn_basic.eg none

We can see that the agent has problems in some situations. Nonetheless, it still tries to follow the same strategy: tracking the ball. Many times it succeeds, and it still represents a serious competition for a human player. Hence, the agent was fairly able to generalize what it learned under different circumstances.

## Conclusion

In this project, I designed and implemented an AI agent for the Pong game. Using Q learning, it learned how to control the paddle in order to avoid missing the ball. I used a neural network approximator for the Q function and the RPROP algorithm to train it. All code was implemented in Java (see the `java_basic` folder).

Recall the questions that I posed in the introduction. Here are my conclusions as answers to those questions:

<b>Can an agent that is trained using Q learning behave well (in a robust and stable manner) in a simplified version of Pong?</b>

Yes. The most difficult challenge I came upon was finding a good set of learning parameters. This difficulty was overcome largely by trial and error. Once these parameters were found, the agent was able to learn reasonable behaviors and achieve a high performance (in terms of the average distance by which it misses the ball). Most remarkably, it was able to learn to follow the ball.

<b>How complex does the neural network approximator have to be in order to obtain good learning performance?</b>

Based on my trial and error experimentation, the neural network needs to be fairly complex in terms of depth in order to learn fast. The neural network I used had six hidden layers with a total of 54 hidden units.

<b>How does the behavior displayed by the trained controller compare to the behavior displayed by a human player?</b>

As you probably experienced with the commands above, the agent is very much able to compete against a human.

<b>Can the agent transfer what it learned in order to play under slightly different game dynamics?</b>

Yes. I tried to increase the speed of the ball slightly and use the agent with the neural network that was trained with the old speed. Even though the agent had troubles catching up with the ball in some situations, it still followed the same general strategy it learned: tracking the ball. In fact it was still competition for a human player.

As future work, I would like to implement the missing gameplay rule: the bouncing angle changes depending on where the ball hit the paddle.